## Evolutionary Algorithms Demo

The data for this demo is the same as seen previously (i.e., sexual clinic location problem).

### Imports 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# you will use itertools for enumerating all solutions in small instances.
from itertools import combinations

###  `metapy` package imports

The package contains the classes and functions for the evolutionary algorithms you will use in this notebook.

In [3]:
from metapy.evolutionary.evolutionary import (EvolutionaryAlgorithm, 
                                              MuLambdaEvolutionStrategy, 
                                              MuPlusLambdaEvolutionStrategy,
                                              GeneticAlgorithmStrategy,
                                              ElitistGeneticAlgorithmStrategy,
                                              WeightedAverageObjective,
                                              FacilityLocationPopulationGenerator,
                                              BasicFacilityLocationMutator,
                                              TournamentSelector,
                                              FacilityLocationSinglePointCrossOver)

Function to determine the number of possible combinations...

In [4]:
def combination_counter(n_facilities, p):
    facility = np.arange(n_facilities, dtype=np.uint8)
    combs = [np.array(a) for a in combinations(facility, p)]
    combs_len = len(combs)
    combs_len = format(combs_len, ",")
    print(f"Placing {p} facilities from a possible {n_facilities} " +
          f"location yields {combs_len} possible combinations")

### Import case study data

The car travel times in minutes from annoymised postcode sectors to annoymised clinic locations.

In [5]:
travel_matrix = pd.read_csv('../data/clinic_car_travel_time.csv', 
                            index_col='sector')
travel_matrix.head()

,clinic_1,clinic_2,clinic_3,clinic_4,clinic_5,clinic_6,clinic_7,clinic_8,clinic_9,clinic_10,...,clinic_19,clinic_20,clinic_21,clinic_22,clinic_23,clinic_24,clinic_25,clinic_26,clinic_27,clinic_28
sector,,,,,,,,,,,,,,,,,,,,,
PS158,33.17,40.15,38.17,37.93,29.35,51.48,53.28,48.00,53.82,44.65,...,12.10,12.27,15.83,53.27,53.98,29.75,34.22,32.68,19.62,39.25
PS159,31.42,36.55,36.42,34.53,27.60,47.88,49.68,44.40,50.22,41.05,...,11.75,11.92,10.62,49.68,50.38,26.15,30.62,32.35,19.28,35.65
PS160,31.82,38.80,36.82,36.58,28.00,50.13,51.95,46.65,52.47,43.30,...,10.75,10.92,14.35,51.93,52.65,28.40,32.87,31.35,18.27,37.90
PS161,31.68,38.65,36.67,36.43,27.87,49.98,51.80,46.50,52.32,43.17,...,10.32,10.77,16.38,51.78,52.50,28.27,32.73,31.20,17.82,37.75
PS162,29.55,36.53,34.55,34.32,25.73,47.87,49.67,44.38,50.20,41.03,...,6.77,7.28,17.18,49.65,50.37,26.13,30.60,29.07,14.27,35.63


In [6]:
# no of cases by postcode sector...

cases = pd.read_csv('../data/sh_demand.csv', index_col='sector')
cases.head()

,n_patients
sector,
PS1,3375
PS2,3338
PS3,2922
PS4,3191
PS5,3134


#### 1. Generating an initial population.

The first task when using a population based method is to create an initial random population of solutions!  For our purposes, this is a multi-dimensional array.  We can use an object of type `FacilityLocationPopulationGenerator` to do the work for us here.

```python
from metapy.evolutionary.evolutionary import FacilityLocationPopulationGenerator
```

`FacilityLocationPopulationGenerator` accepts three arguments when it is created:

* `n_candidates`: int.  This is $P$ the number of candidate locations
* `n_facilities`: int. This is $p$ the number of facilities to place.
* `random_seed`: int, optional (default=None).  Set if you want a reproducible result.  For example = 42.

`FacilityLocationPopulationGenerator` has a single method `generate` that accepts a parameter specifying the population size.  It returns a multi-dimensional numpy array.

Let's assume you want have a problem with $P$ = 28, $p$ = 8 and we want to create a population of size 10.

```python
#example solution
N_CANDIDATES = 28
N_FACILITIES = 8
SEED = 42
POPULATION_SIZE = 10

gen = FacilityLocationPopulationGenerator(n_candidates=N_CANDIDATES,
                                          n_facilities=N_FACILITIES,
                                          random_seed=SEED)


gen.generate(population_size=POPULATION_SIZE)
```

In [7]:
combination_counter(28, 8)

Placing 8 facilities from a possible 28 location yields 3,108,105 possible combinations


In [8]:
# example solution
N_CANDIDATES = 28
N_FACILITIES = 8
SEED = 42
POPULATION_SIZE = 10

# Instantiate the FacilityLocationPopulationGenerator class into an object variable (gen)
gen = FacilityLocationPopulationGenerator(n_candidates=N_CANDIDATES,
                                          n_facilities=N_FACILITIES,
                                          random_seed=SEED)

# Remember the 'gen' object as a single method generate 
# that accepts a parameter specifying the population size. 
gen.generate(population_size=10)

array([[ 2, 19, 22, 15, 24, 10,  1, 17],
       [24, 16, 11, 10,  2, 20, 13,  5],
       [23,  1,  4, 27, 13,  9, 22,  2],
       [ 5,  9, 19, 25, 22, 20, 10, 16],
       [ 8, 24, 25, 16, 27, 10, 23, 19],
       [24, 13,  5,  7,  9,  4, 16, 11],
       [19, 17, 27, 23,  6, 22,  9,  7],
       [17, 15,  7, 24, 16, 21, 27, 11],
       [25,  2, 14,  9, 15, 11, 27, 21],
       [13,  7,  0, 10,  6, 25,  5,  8]], dtype=int8)

#### 2: Mutating a solution

* Basic evolutionary strategies work by mutating the most promising solutions in the population.  
* There are many ways to implement mutation.  
* Here you will use `BasicFacilityLocationMutator`.  
* Each element in a solution has a constant probability of mutation (by default 1 / no. of facilities in a solution, but you may wish to set this higher.).  
* If a facility is chosen then it is replaced by a random facility not currently in the solution.

You can create a `BasicFacilityLocationMutator` as follows:

```python
mutator = BasicFacilityLocationMutator(n_candidates=28,
                                       solution_size=4)
solution = np.array([1, 2, 3, 4])

mutant = mutator.mutate(solution)
print(mutant)
```

To use a higher mutation rate:

```python
mutator = BasicFacilityLocationMutator(n_candidates=28,
                                  solution_size=4,
                                  mutation_rate=0.6)
solution = np.array([1, 2, 3, 4])

mutant = mutator.mutate(solution)
print(mutant)
```

In [6]:
# mutating through 10 generations
gens = 10


mutator = BasicFacilityLocationMutator(n_candidates=28,
                                       solution_size=4,
                                       mutation_rate=0.5)
solution = np.arange(5)

for i in range(gens):
    solution = mutator.mutate(solution)
    print(f'solution {sorted(solution)}')

solution [0, 2, 3, 4]
solution [0, 1, 4, 19]
solution [1, 4, 13, 19]
solution [1, 13, 19, 21]
solution [13, 19, 21, 23]
solution [6, 10, 21, 23]
solution [10, 17, 18, 23]
solution [10, 17, 18, 22]
solution [19, 24, 25, 26]
solution [10, 12, 16, 25]


#### Exercise 3: The <span style="color:red"> $(\mu, \lambda)$ </span> and <span style="color:blue">$(\mu+\lambda)$ </span> evolutionary strategies

A random initial population and a mutation operator provide the ingredients for the two basic evolutionary strategies: <span style="color:red"> $(\mu, \lambda)$ </span> and <span style="color:blue">$(\mu+\lambda)$ </span>.

Remember...
* $(\lambda)$ "lambda" represents initial population size (i.e. how many solutions)
* $(\mu)$ "mu" represents the number of best solutions to keep, remainder are deleted

Below you will :
* Run two evolutionary algorithms with <span style="color:red">$(\mu, \lambda)$</span> and <span style="color:blue">$(\mu$ **+** $\lambda)$</span>  strategies respectively.
* Investigate the parameters requried when creating <span style="color:red">**MuLambdaEvolutionStrategy**</span>  and <span style="color:blue"> **MuPlusLambdaEvolutionStrategy** </span> 
* Use a problem size of 28 candidate locations and 14 facilities
* Initially try 
 * $\mu $("mu" aka number of best solutions to keep) = 10; and 
 * $\lambda$ ("lambda" aka initial population size) = 200. 
* Using a random initial population evolve for 50 generations.

**Note**:
* Evolutionary strategies are computationally expensive.  Expect a 50 generation algorithm to take 20-45 seconds on your machine.

Read about `%%time` [here](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-time)

In [16]:
%%time

# Evolutionary Algorithm - (mu,lambda) strategy
n_candidates = 28
n_facilities = 14

mu = 10
_lambda = 200

# objective
objective = WeightedAverageObjective(cases, travel_matrix)

# initial solution generator
init = FacilityLocationPopulationGenerator(n_candidates, n_facilities)

# mutation operator
mutator = BasicFacilityLocationMutator(n_candidates=n_candidates, 
                                       solution_size=n_facilities, 
                                       mutation_rate=0.2, verbose=False)

# evolutionary stategy - mu, lambda
strategy = MuLambdaEvolutionStrategy(mu, _lambda, mutator)

# solver object
solver = EvolutionaryAlgorithm(init, objective, _lambda, strategy, 
                               maximisation=False, generations=50)

print("\nRunning (mu, lambda) evolutionary alg...")
solver.solve()

print("\n** (MU,LAMBDA) OUTPUT ***")
print("best cost:\t{0}".format(solver.best_fitness))
print("best solutions:")
print(sorted(solver.best_solution))
print("\n ------")


Running (mu, lambda) evolutionary alg...

** (MU,LAMBDA) OUTPUT ***
best cost:	5.8758212431381835
best solutions:
[0, 1, 3, 4, 6, 7, 8, 10, 11, 15, 17, 22, 24, 26]

 ------
CPU times: user 10.9 s, sys: 7.95 ms, total: 10.9 s
Wall time: 10.9 s


In [17]:
%%time

# example solution

# Evolutionary Algorithm - (mu+lambda) strategy
n_candidates = 28
n_facilities = 14

mu = 10
_lambda = 200

# objective
objective = WeightedAverageObjective(cases, travel_matrix)

# initial solution generator
init = FacilityLocationPopulationGenerator(n_candidates, n_facilities)

# mutation operator
mutator = BasicFacilityLocationMutator(n_candidates=n_candidates, 
                                       solution_size=n_facilities, 
                                       mutation_rate=0.2, verbose=False)

# evolutionary stategy - mu PLUS lambda
strategy = MuPlusLambdaEvolutionStrategy(mu, _lambda, mutator)

# solver object
solver = EvolutionaryAlgorithm(init, objective, _lambda, strategy, 
                               maximisation=False, generations=50)

print("\nRunning (mu + lambda) evolutionary alg...")
solver.solve()

print("\n** (MU+LAMBDA) OUTPUT ***")
print("best cost:\t{0}".format(solver.best_fitness))
print("best solutions:")
print(sorted(solver.best_solution))
print("\n ------")


Running (mu + lambda) evolutionary alg...

** (MU+LAMBDA) OUTPUT ***
best cost:	5.8758212431381835
best solutions:
[0, 1, 3, 4, 6, 7, 8, 10, 11, 15, 17, 22, 24, 26]

 ------
CPU times: user 11.4 s, sys: 21 µs, total: 11.4 s
Wall time: 11.4 s


Click here to see random restarts!!!!!!!

# Exercise 4: Locating facilities using a full Genetic Algorithm (GA)

Now that you have warmed up using  $(\mu, \lambda)$ and $(\mu+\lambda)$  it is time to move onto a full GA.  This means you need to take account of two further steps.

* A selection operator for breeding - in this instance you will use the provided `TournamentSelector`
* A crossover operator for breeding - you will use `FacilityLocationSinglePointCrossover`

See lecture slides for details of how these work.

`metapy` provides standard and elitist GA strategies.  The code provided in this task demonstrates how these are instantiated and used to solve the sexual health clinic facility location problem.

**Task:**

The two code blocks below have been provided to demonstrate how to use run the `metapy` implementations of a GA.  Note that these are similar to the two basic evolutionary strategies you used in the previous exercise.  

* Run `GeneticAlgorithmStrategy` and `ElitistGeneticAlgorithmStrategy` using the parameters provided


**Questions**:

* Are you satisfied with the results?  If not you could try changing/tuning the parameters:
    * lambda and mu
    * the number of generations (note this will start to get slow with large numbers)
    * the mutation rate
    * (You could also work with a smaller problem size to speed things up).
    
* Which of the algorithms used fin this case study do you prefer the most and why?

* Ultimately, in practice, if you are not satisfied with the performance of the GA you might need to code new cross-over and mutation operators! 

    

In [9]:
%%time

# Evolutionary Algorithm - Genetic Algorithm strategy

n_candidates = 28
n_facilities = 14

_lambda = 200
# objective
objective = WeightedAverageObjective(cases, travel_matrix)

# initial solution generator
init = FacilityLocationPopulationGenerator(n_candidates, n_facilities)

# mutation operator
mutator = BasicFacilityLocationMutator(n_candidates=n_candidates, 
                                       solution_size=n_facilities, 
                                       mutation_rate=0.2, verbose=False)

# cross over operator
x_over = FacilityLocationSinglePointCrossOver()

#GA strategy
strategy = GeneticAlgorithmStrategy(_lambda, 
                                    selector=TournamentSelector(),
                                    xoperator=x_over,
                                    mutator=mutator)


solver = EvolutionaryAlgorithm(init, objective,_lambda, strategy, 
                               maximisation=False, generations=50)
print("\nRunning Genetic Algorithm")
solver.solve()

print("\n** GA OUTPUT ***")
print("best cost:\t{0}".format(solver.best_fitness))
print("best solutions:")
print(solver.best_solution)


Running Genetic Algorithm

** GA OUTPUT ***
best cost:	6.129377770191772
best solutions:
[25  3  0  9 27 26  7 16 12 17 11 15 22  8]
CPU times: user 11.4 s, sys: 3.94 ms, total: 11.4 s
Wall time: 11.4 s


In [10]:
%%time 

# Evolutionary Algorithm - Elistist Genetic Algorithm strategy

n_candidates = 28
n_facilities = 14

# GA parameters
mu = 10
_lambda = 200

# objective
objective = WeightedAverageObjective(cases, travel_matrix)

# initial solution generator
init = FacilityLocationPopulationGenerator(n_candidates, n_facilities)

# mutation operator
mutator = BasicFacilityLocationMutator(n_candidates=n_candidates, 
                                       solution_size=n_facilities, 
                                       mutation_rate=0.2, verbose=False)

# cross over operator
x_over = FacilityLocationSinglePointCrossOver()

# GA strategy
strategy = ElitistGeneticAlgorithmStrategy(mu,
                                           _lambda, 
                                           selector=TournamentSelector(),
                                           xoperator=x_over,
                                           mutator=mutator)


solver = EvolutionaryAlgorithm(init,
                               objective,
                               _lambda,
                               strategy, 
                               maximisation=False, generations=50)
print("\nRunning Elitist Genetic Algorithm")
solver.solve()

print("\n** ELITIST GA OUTPUT ***")
print("best cost:\t{0}".format(solver.best_fitness))
print("best solutions:")
print(solver.best_solution)


Running Elitist Genetic Algorithm

** ELITIST GA OUTPUT ***
best cost:	6.389238028642063
best solutions:
[22  2  8  6 10  0  4 16 11  7 25 17 26 12]
CPU times: user 12.1 s, sys: 7.61 ms, total: 12.1 s
Wall time: 12.1 s


# End